In [2]:
import pandas as pd

In [3]:
import glob

In [3]:
first_cycle=1
for gu_ner_file in glob.glob('extracted_named_entities_output/sampled_*'):
    year=gu_ner_file.split('_')[-2]
    gu_ents_cycle = pd.read_csv(gu_ner_file,index_col=0)
    gu_ents_cycle['article_year']=year
    if first_cycle:
        gu_ents=gu_ents_cycle
        first_cycle=0
    else:
        gu_ents=pd.concat([gu_ents, gu_ents_cycle],ignore_index=True)

In [ ]:
gu_ents=gu_ents[gu_ents['label'].isin(['PERSON','ORG'])]

In [ ]:
gu_ents['article_year']=gu_ents['article_year'].astype(int)

In [7]:
gu_ents.drop(['label','start','end','start_char','end_char'],1,inplace=True)

/tmp/ipykernel_2382/1783612290.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  gu_ents.drop(['label','start','end','start_char','end_char'],1,inplace=True)


In [8]:
gu_ents.drop_duplicates(inplace=True)

In [9]:
gu_ents.shape

(3552188, 3)

In [10]:
gu_ents['text'].nunique()

777413

In [11]:
entity_counts=gu_ents['text'].value_counts()

In [12]:
entity_counts

Guardian                            26181
EU                                  10401
Trump                                9900
Labour                               9219
BBC                                  8569
                                    ...  
Dougie Evans’s                          1
Stankevicius                            1
Darius A Stankevicius                   1
Ieva Kuniskis                           1
Queen Mamohato Memorial hospital        1
Name: text, Length: 777413, dtype: int64

In [13]:
gu_ents_low_prevalence=gu_ents[gu_ents['text'].isin(entity_counts[entity_counts<=3].index.values)]

In [14]:
gu_ents_high_prevalence=gu_ents[gu_ents['text'].isin(entity_counts[entity_counts>3].index.values)]

In [15]:
del(gu_ents)

In [18]:
gu_ents_high_prevalence['text'].nunique()

106944

In [ ]:
unique_ents=set(gu_ents_high_prevalence['text'])

In [3]:
cycle=0
for entities in zip(*(iter(unique_ents),) * 1000):
    entities_df=gu_ents_high_prevalence[gu_ents_high_prevalence['text'].isin(entities)]
    entities_df.groupby('text').apply(lambda x: x.sample(n=3, replace=False)).reset_index(drop = True)
    if cycle==0:
        downsampled_gu_ents_high_prevalence=entities_df
    else:
        downsampled_gu_ents_high_prevalence=pd.concat([downsampled_gu_ents_high_prevalence,entities_df], ignore_index=True)
    cycle+=1

NameError: name 'unique_ents' is not defined

In [30]:
resampled_gu_articles=pd.concat([downsampled_gu_ents_high_prevalence.sample(5000),gu_ents_high_prevalence.sample(5000)])

In [32]:
resampled_gu_articles.drop(['text'],1,inplace=True)

/tmp/ipykernel_2382/1973138111.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  resampled_gu_articles.drop(['text'],1,inplace=True)


In [37]:
resampled_gu_articles.drop_duplicates(inplace=True)

In [39]:
resampled_gu_articles

,doc_index,article_year
23512,6433,2022
346671,6308,2015
2658567,1929,2021
486947,4716,2017
1415889,9013,2016
...,...,...
1023360,4105,2018
4142860,11203,2014
4220730,12378,2014
746114,9788,2022


In [4]:
gu_body_texts=''
first_cycle=1
for gu_file in glob.glob('sampled_*'):
    year=gu_file.split('_')[-1].split('.')[-2]
    gu_article=pd.read_csv(gu_file)
    #gu_article=gu_article[['body_text']]
    gu_article['article_year']=int(year)
    gu_article['doc_index']=gu_article.index.values
    gu_article=gu_article.merge(resampled_gu_articles)
    if first_cycle:
        gu_body_texts=gu_article
        first_cycle=0
    else:
        gu_body_texts=pd.concat([gu_body_texts, gu_article],ignore_index=True)

NameError: name 'resampled_gu_articles' is not defined

In [ ]:
gu_body_texts

In [52]:
gu_body_texts.to_csv('gu_resampled.csv')

In [4]:
gu_body_texts=''
first_cycle=1
for gu_file in glob.glob('sampled_*'):
    year=gu_file.split('_')[-1].split('.')[-2]
    gu_article=pd.read_csv(gu_file)
    #gu_article=gu_article[['body_text']]
    gu_article['article_year']=int(year)
    gu_article['doc_index']=gu_article.index.values
    if first_cycle:
        gu_body_texts=gu_article
        first_cycle=0
    else:
        gu_body_texts=pd.concat([gu_body_texts, gu_article],ignore_index=True)

In [23]:
gu_body_texts.dropna(subset=['body_text'],inplace=True)

In [24]:
gu_body_texts.shape

(183158, 16)

In [6]:
gu_body_texts.head()

,path,headline,url,content_type,section_id,pillar_id,web_publication_date,word_count,trail_text,production_office,byline,body_text,body_html,keyword_tag,article_year,doc_index
0,/law/commentisfree/2022/mar/26/jurors-who-sat-...,Jurors who sat in the Zachary Rolfe murder tri...,www.theguardian.com/law/commentisfree/2022/mar...,Article,law,pillar/news,2022-03-25 22:24:09+00:00,1114,"In criminal trials, the information that is ex...",Aus,Richard Ackland,Jurors who sat on the Zachary Rolfe murder tri...,<p>Jurors who sat on the Zachary Rolfe murder ...,['law/law-australia' 'australia-news/australia...,2022,0
1,/crosswords/crossword-blog/2022/jul/04/crosswo...,Crossword roundup: could your puzzle find a ho...,www.theguardian.com/crosswords/crossword-blog/...,Article,crosswords,pillar/lifestyle,2022-07-04 11:20:36+00:00,973,"A friendly community with a lot of puzzles, wh...",Uk,Alan Connor,I was delighted to read about New York Times s...,<p>I was delighted to read about New York Time...,['crosswords/crosswords' 'lifeandstyle/hobbies...,2022,1
2,/retail-reimagined/2022/jan/13/how-businesses-...,‘There’s genuine value in having a nice experi...,www.theguardian.com/retail-reimagined/2022/jan...,Article,retail-reimagined,NaN,2022-01-13 15:53:41+00:00,804,"Frictionless payments, ease of use and persona...",Uk,Duncan Jefferies,"As consumers, we have increasingly grown more ...","<p>As consumers, we have increasingly grown mo...",[],2022,2
3,/law/2022/apr/29/barrister-allison-bailey-ston...,Barrister was discriminated against for gender...,www.theguardian.com/law/2022/apr/29/barrister-...,Article,law,pillar/news,2022-04-29 12:23:43+00:00,621,Allison Bailey says Garden Court chambers and ...,Uk,Haroon Siddique Legal affairs correspondent,A barrister was unlawfully discriminated again...,<p>A barrister was unlawfully discriminated ag...,['law/employment-law' 'law/law' 'world/gender'...,2022,3
4,/law/2022/jun/17/sonia-sotomayor-supreme-court...,Sonia Sotomayor says supreme court’s ‘mistakes...,www.theguardian.com/law/2022/jun/17/sonia-soto...,Article,law,pillar/news,2022-06-17 09:00:06+00:00,483,Liberal-leaning justice says ‘there are days I...,Us,Guardian staff and agencies,The liberal-leaning supreme court justice Soni...,<p>The liberal-leaning supreme court justice S...,['law/us-supreme-court' 'us-news/us-news'],2022,4


In [7]:
gu_body_texts['content_type'].unique()

array(['Article', 'Interactive', 'Liveblog'], dtype=object)

In [9]:
gu_body_texts['section_id'].unique()

array(['law', 'crosswords', 'retail-reimagined', 'theobserver',
       'follow-your-passion', 'info', 'inequality', 'the-whole-picture',
       'what-if-in-california', 'make-your-world-work',
       'seek-the-new-world-of-work-', 'supercharge-your-business',
       'guardian-masterclasses', 'sustainable-whisky', 'reimagining-work',
       'dear-evan-hansen-the-21st-century-musical', 'scotlands-stories',
       'global', 'huelwear-clothes-for-the-conscious',
       'going-global-with-business', 'games',
       'observer-food-monthly-awards', 'sustainable-home-improvements',
       'further-education-share-your-skills',
       'the-time-of-your-life-in-croatia', 'spotify-morning-moods',
       'the-only-parmesan', 'amc-only-the-good-stuff',
       'connected-thinking', 'my-sherry-amour', 'knowledge-is-pleasure',
       'gnm-press-office', 'mirvac-voyager', 'cities',
       'mg-motor-switch-to-electric', 'plan-the-perfect-ski-break',
       'all-win', 'snooze-investing-in-sleep', 'welcom

In [10]:
gu_body_texts['pillar_id'].unique()

array(['pillar/news', 'pillar/lifestyle', nan, 'pillar/arts',
       'pillar/sport', 'pillar/opinion'], dtype=object)

In [32]:
gu_body_texts=gu_body_texts[gu_body_texts['section_id'].isin(['law', 'global','global-development', 'news', 'world', 'uk-news', 'us-news', 'business','politics'])]


In [34]:
gu_body_texts.to_csv('gu_resampled_by_section_id.csv')